# Premium Allocation Tool

In [ ]:
import numpy as np
import pandas as pd
import json
import uuid  

from pat_back import PatJob, PatFlag, AppSettings, PatHelper

# Get job list 


## Delete Job

In [ ]:
# pd.DataFrame(json.loads(PatJob.get_job_list()))
lst= [a for a in range(100000233,100000300)]
PatHelper.delete(*lst)

## Job parameters

In [ ]:
ret = PatJob.get_job_para(100000197)
if ret:
    print(json.dumps(json.loads(ret),indent=4))


## Result summary 

In [ ]:
df = PatJob.get_summary(100000197)
df

## Results

In [ ]:
df = PatJob.get_results(100000197)
df

## Validation data

In [ ]:
df1,df2,df3 = PatJob.get_validation_data(100000197)
print(len(df1), len(df2), len(df3))

In [ ]:
df2.sort_values(by='LocationIDStack')

# Create New Job 

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid']=str(uuid.uuid4())

job = PatJob(job_para)
job.job_id

# Process

In [ ]:
job.perform_analysis()

# Testing

In [ ]:

import json
import numpy as np
import pandas as pd

import pyodbc
from bcpandas import SqlCreds, to_sql

df = pd.read_csv('data/psold_aoi.csv')

creds = SqlCreds(AppSettings.PAT_JOB_SVR, AppSettings.PAT_JOB_DB, AppSettings.PAT_JOB_USR, AppSettings.PAT_JOB_PWD)
to_sql(df,'psold_aoi', creds, index=False, if_exists='append') 

In [ ]:
job_conn = f'''DRIVER={{SQL Server}};Server={AppSettings.PAT_JOB_SVR};Database={AppSettings.PAT_JOB_DB};
            User Id={AppSettings.PAT_JOB_USR};Password={AppSettings.PAT_JOB_PWD};
            MultipleActiveResultSets=true;'''
with pyodbc.connect(job_conn) as conn:
             dfHPR = pd.read_sql_query(f"""select * from psold_hpr order by limit""", conn)
dfHPR

# Start

In [7]:
from datetime import datetime
import numpy as np
import pandas as pd

from pat_back import Psold

df = Psold.sample_pat(hpr=False, curveType='net')
df.sort_values('locID')[['locID','polID','locID','allocPrem']]


,locID,polID,locID,allocPrem
0,1,1,1,2.011357e+05
1,2,2,2,1.737827e+05
2,3,3,3,7.697372e+05
3,4,4,4,0.000000e+00
4,5,5,5,8.579737e+04
5,6,6,6,2.133823e+06
6,7,7,7,4.413706e+06
7,8,8,8,3.364052e+05
8,9,9,9,1.043745e+04
9,10,10,10,2.000000e+05
